In [29]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies, POMDPModelTools
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock
using NativeSARSOP


## Initialization

In [30]:
global Number_level = 4;
global Action_num = 4;
gamma=0.95

States=Array((1:Number_level))
Observations=Array((1:Number_level))


4-element Vector{Int64}:
 1
 2
 3
 4

In [31]:
Observation_matrix=[
    0.8 0.2 0.0 0.0;
    0.1 0.8 0.1 0.0;
    0.0 0.2 0.8 0.0;
    0.0 0.0 0.0 1.0
]


4×4 Matrix{Float64}:
 0.8  0.2  0.0  0.0
 0.1  0.8  0.1  0.0
 0.0  0.2  0.8  0.0
 0.0  0.0  0.0  1.0

In [32]:
pdf(Categorical(Observation_matrix[1,:]),2)

0.2

In [33]:
Observation_matrix[1,2]

0.2

In [34]:
T=zeros(Number_level,Number_level,3,100);

# Load transition probability matrix for each component
for i in 1:100
    fullname = "C:/Users/hhu98/OneDrive - UW-Madison/POMDP/code/Heterogeneous system/hetero_tm/tm"*string(i);
   
    Transition_matrix = load(fullname*".jld2","transition_matrix");
    # the last term in argument of T, which is written as 'i' here,
    # is the index of component in the heterogeneous system
    T[:,:,1,i].=Transition_matrix; # a=1: do not replace
    T[:,:,2,i].=Transition_matrix; # a=2: corrective replacement
    for j in 1:Number_level
        T[j,:,3,i].=Transition_matrix[1,:]; # a=3: preventive replacement
        end
    # T[Number_level,:,1,i].=Transition_matrix[1,:]
    T[Number_level,:,1,i].=0.0
    T[Number_level,Number_level,1,i]=1.0
    T[Number_level,:,2,i].=Transition_matrix[1,:]

end

In [35]:
# T[:,:,:,30]

In [36]:
NumberUnits=20
limit=12
failure_penalty = -1200.0;
maintenance_penalty = -100.0;
setup_cost = -300.0;
normal_operation = 0.0;
system_penalty=-2000

-2000

In [37]:
Initialstates = zeros(Number_level)
Initialstates[1]=1.0
Initialstates

4-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0

In [38]:
Initialstate = SparseCat(States, Initialstates)

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 1.0   
   2 ┤ 0.0                                      
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [39]:
Initialobs = SparseCat(Observations, Observation_matrix[1,:])

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

## Define POMDPs

In [40]:
function create_pomdps()

Unit_pomdps = Dict{Int,POMDP}()

for i in 1:NumberUnits

 Unit_pomdps[i]= QuickPOMDP(
    states=States,
    actions=Array((1:Action_num)),
    observations = Observations,
    initialstate = Initialstate,
    initialobs = Initialobs,
    discount = gamma,

    transition = function(s,a)
        if a == 1 # NO REPAIR, NO SYSTEM FAILURE
            return SparseCat(States, T[s,:,1,i])
        end
        if a == 2 # UNIT NOT REPAIRED UNLESS FAILURE BUT REPAIR IN SYSTEM
            if s == Number_level
                s = 1
                return SparseCat(States, T[s,:,2,i])
            else
                return SparseCat(States, T[s,:,2,i])
            end
        end
        if a == 3 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM, NO SYSTEM FAILURE
            if s==Number_level
                s=1
                return SparseCat(States, T[s,:,3,i])
            else
                s = 1
                return SparseCat(States, T[s,:,3,i])
            end
        end
        if a == 4 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM,SYSTEM FAILED
            if s==Number_level
                s=1
                return SparseCat(States, T[s,:,3,i])
            else
                s = 1
                return SparseCat(States, T[s,:,3,i])
            end
        end
    end,

    observation = function(a,s)
        return SparseCat(Observations, Observation_matrix[s,:])
    end,
        
    reward = function (s, a)
        if a == 1 # NO REPAIR, NO SYSTEM FAILURE
            if s == Number_level
                return system_penalty/(limit)
            else
                return normal_operation
            end
        end
        if a == 2 # UNIT NOT REPAIRED UNLESS FAILURE BUT REPAIR IN SYSTEM, NO SYSTEM FAILURE PENALTY
            if s == Number_level
                return failure_penalty +  setup_cost/NumberUnits + normal_operation
            else
                # Here it doesn't matter whether systems fails or not, since C_b does not apply to functional units
                return normal_operation +  setup_cost/NumberUnits
            end
        end
        if a == 3 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM, NO SYSTEM FAILURE PENALTY
            if s==Number_level
                return failure_penalty +  setup_cost/NumberUnits + normal_operation
            else
                # Here it doesn't matter whether systems fails or not, since C_b does not apply to functional units
                return maintenance_penalty +  setup_cost/NumberUnits+ normal_operation 
            end
        end
        if a == 4 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM,SYSTEM FAILED
            if s==Number_level
                return failure_penalty +  setup_cost/NumberUnits + normal_operation + system_penalty/(limit)
            else
                return maintenance_penalty +  setup_cost/NumberUnits + normal_operation
            end
        end
    end,
    
)
end
return Unit_pomdps
end

create_pomdps (generic function with 1 method)

## Solve POMDPs

In [41]:
function solve_pomdps(pomdps)
    policies = Dict{Int, Any}()
    
    for (idx, pomdp) in pomdps
        solver = SARSOPSolver()
        policies[idx] = solve(solver, pomdp)
    end
    
    return policies
end

solve_pomdps (generic function with 1 method)

In [42]:
Unit_pomdps = create_pomdps()

Dict{Int64, POMDP} with 20 entries:
  5  => QuickPOMDP{UUID("f5c2a699-e725-4e8f-b252-adbcb81d2cd7"), Int64, Int64, …
  16 => QuickPOMDP{UUID("e317cb13-40c0-4a43-8da3-af6783205611"), Int64, Int64, …
  20 => QuickPOMDP{UUID("dab042e0-f58d-4241-a6e9-1b8b0d94cb07"), Int64, Int64, …
  12 => QuickPOMDP{UUID("be62d620-32db-422c-abd2-6eb238c8e9fa"), Int64, Int64, …
  8  => QuickPOMDP{UUID("51f30ebe-7915-4848-9c5f-7838fec66cdb"), Int64, Int64, …
  17 => QuickPOMDP{UUID("34b32356-48c6-424f-975d-521b13de1acd"), Int64, Int64, …
  1  => QuickPOMDP{UUID("b427c234-47a3-4735-96bb-47c4694b9213"), Int64, Int64, …
  19 => QuickPOMDP{UUID("08fca380-034a-497b-b39d-3698314c8d51"), Int64, Int64, …
  6  => QuickPOMDP{UUID("d812378d-f570-4763-9f8b-dc104c092b52"), Int64, Int64, …
  11 => QuickPOMDP{UUID("1fbcd60b-bc2a-4b33-aca4-a789aa737560"), Int64, Int64, …
  9  => QuickPOMDP{UUID("77f21481-ec52-4ac5-8c1d-658e08cb40a6"), Int64, Int64, …
  14 => QuickPOMDP{UUID("adf7883d-b88b-413b-98f9-c1b15c62ea1d"), Int64, I

In [43]:
transition(Unit_pomdps[1],1,1)

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■ 0.846701689363337   
   2 ┤■■■■ 0.153298310636663                    
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [44]:
pdf(transition(Unit_pomdps[1],1,1),1), pdf(transition(Unit_pomdps[1],1,1),2)


(0.846701689363337, 0.153298310636663)

In [45]:
T[1,1,1,1], T[1,2,1,1]

(0.846701689363337, 0.153298310636663)

In [46]:
solver = SARSOPSolver()
policy = solve(solver, Unit_pomdps[1])
# Calling observation

AlphaVectorPolicy{QuickPOMDP{Base.UUID("b427c234-47a3-4735-96bb-47c4694b9213"), Int64, Int64, Int64, @NamedTuple{stateindex::Dict{Int64, Int64}, isterminal::Bool, obsindex::Dict{Int64, Int64}, states::Vector{Int64}, observations::Vector{Int64}, discount::Float64, initialobs::SparseCat{Vector{Int64}, Vector{Float64}}, actions::Vector{Int64}, observation::var"#17#20", actionindex::Dict{Int64, Int64}, initialstate::SparseCat{Vector{Int64}, Vector{Float64}}, transition::var"#16#19"{Int64}, reward::var"#18#21"}}, Int64}(QuickPOMDP{Base.UUID("b427c234-47a3-4735-96bb-47c4694b9213"), Int64, Int64, Int64, @NamedTuple{stateindex::Dict{Int64, Int64}, isterminal::Bool, obsindex::Dict{Int64, Int64}, states::Vector{Int64}, observations::Vector{Int64}, discount::Float64, initialobs::SparseCat{Vector{Int64}, Vector{Float64}}, actions::Vector{Int64}, observation::var"#17#20", actionindex::Dict{Int64, Int64}, initialstate::SparseCat{Vector{Int64}, Vector{Float64}}, transition::var"#16#19"{Int64}, reward

In [47]:
policies = Dict()

Dict{Any, Any}()

In [48]:
policies[1] = policy

AlphaVectorPolicy{QuickPOMDP{Base.UUID("b427c234-47a3-4735-96bb-47c4694b9213"), Int64, Int64, Int64, @NamedTuple{stateindex::Dict{Int64, Int64}, isterminal::Bool, obsindex::Dict{Int64, Int64}, states::Vector{Int64}, observations::Vector{Int64}, discount::Float64, initialobs::SparseCat{Vector{Int64}, Vector{Float64}}, actions::Vector{Int64}, observation::var"#17#20", actionindex::Dict{Int64, Int64}, initialstate::SparseCat{Vector{Int64}, Vector{Float64}}, transition::var"#16#19"{Int64}, reward::var"#18#21"}}, Int64}(QuickPOMDP{Base.UUID("b427c234-47a3-4735-96bb-47c4694b9213"), Int64, Int64, Int64, @NamedTuple{stateindex::Dict{Int64, Int64}, isterminal::Bool, obsindex::Dict{Int64, Int64}, states::Vector{Int64}, observations::Vector{Int64}, discount::Float64, initialobs::SparseCat{Vector{Int64}, Vector{Float64}}, actions::Vector{Int64}, observation::var"#17#20", actionindex::Dict{Int64, Int64}, initialstate::SparseCat{Vector{Int64}, Vector{Float64}}, transition::var"#16#19"{Int64}, reward

In [49]:
policy1 = policies[1]
b = [0.1,0.1,0.8,0.0]
value(policy1,b)

-533.5709685134977

In [50]:
function generative(s,a,i,rng=23) # calculate TRUE COMPONENT-LEVEL reward r_i(s_i,a_i) and get evolving states
    if a==1 # No maintenance in the system, no system failure
        crd=Categorical(T[s,:,1,i])
        s=rand(crd)
        r=normal_operation
        obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
        obs = rand(obs_distribution)
        return (sp=s,r=r,o=obs)
    end

    if a==2 # Maintenance occurs in the system, no system failure, corrective maintenance for the unit
        if s==Number_level
            crd=Categorical(T[1,:,2,i])
            s=rand(crd)
            r=failure_penalty  + normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            crd = Categorical(T[s,:,2,i])
            s = rand(crd)
            r = normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end

    if a==3 # Maintenance occurs in the system, no system failure, preventive maintenance for the unit
        if s==Number_level
            crd=Categorical(T[1,:,3,i])
            s=rand(crd)
            r=failure_penalty  + normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            crd = Categorical(T[s,:,3,i])
            s = rand(crd)
            r = maintenance_penalty+normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end

    if a==4 # maintenance in system, system failure occurs, preventive replacement for the unit
        if s==Number_level
            crd = Categorical(T[1,:,3,i])
            s = rand(crd)
            r=failure_penalty+ normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            r = maintenance_penalty+normal_operation
            crd = Categorical(T[1,:,3,i])
            s = rand(crd)
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end
end

        
        
    

generative (generic function with 2 methods)

#### Belief updater

$$b'(s_j)=P(s_j\vert o,a,b)=\dfrac{P(o\vert s_j,a)\sum_{s_i\in \mathcal{S}}P(s_j\vert s_i,a)b(s_i)}{\sum_{s_j\in \mathcal{S}}P(o\vert s_j,a)\sum_{s_i\in \mathcal{S}}P(s_j\vert s_i,a)b(s_i)}$$

In [51]:
# Input: prior belief, action, observation, index of the component
# Output: updated belief of the component
function update_belief(b,a,o,i) 
    # NOTE: 
    # in input, b is belief from last epoch (before a)
    # a is the last action we took
    # o is the new observation after a
    pomdp = Unit_pomdps[i]
    updated_belief = zeros(length(b))
    if o==4
        updated_belief[Number_level]=1.0
        return updated_belief
    end

    if (a==3) ||(a==4)
    # We assume perfect maintenance and it is know to decision maker(belief updater)
        for sp in 1: Number_level
            p_obs = Observation_matrix[sp,o]
            p_trans = pdf(transition(pomdp,1,1),sp) 
            updated_belief[sp] = p_obs * p_trans
        end
        updated_belief ./= sum(updated_belief)
        return updated_belief
    end
    
    for sp in 1:length(states(pomdp)) 
        # p_observation = POMDPs.observation(pomdp,a,sp) 
        # p_obs = pdf(p_observation,o)
        p_obs = Observation_matrix[sp,o]
        p_trans = 0.0
        for s in 1:length(states(pomdp))
            #trans = transition(pomdp, s,a)
            #p_trans += pdf(transition(pomdp, s,a),sp) * pdf(b, s)
            # p_trans += T[s,sp,a] * pdf(b, s)
            # p_trans += T[s,sp,a,i] * b[s]
            p_trans += pdf(transition(pomdp,s,a),sp) * b[s]
        end
        updated_belief[sp] = p_obs * p_trans
    end
        # Normalize the updated belief state to ensure it sums up to 1
        updated_belief ./= sum(updated_belief)
    return updated_belief
end


update_belief (generic function with 1 method)

### System-level decision making

In [52]:
function decision(o, b)
# initialize system-level action vector
    Q=Qfnc(b) # Q is an N*A matrix for each unit-action pair
    a_final = repeat(1:1,NumberUnits)
    failed_components = 0
    for i in 1:NumberUnits # numerate over units
        if o[i]==Number_level # failure state is self-announcing
            failed_components+=1
        end
    end
    if failed_components >= limit # System fails
        for i in 1:NumberUnits
            if o[i]==Number_level
                a_final[i]=4
            else
                a_final[i]=argmax(a->Q[i,a],[2,3]) # If the unit does not fail, the case is the same as a=2,3
            end
        end
        return a_final
    end

    # Based on belief, compare the expected total rewards of setting up maintenance or not
    D0=0.0
    a0=repeat(1:1,NumberUnits)
    for i in 1:NumberUnits
        D0+=Q[i,1]
    end

    D1=0.0
    a1=repeat(1:1,NumberUnits)
    for i in 1:NumberUnits
        a1[i]=argmax(a->Q[i,a],[2,3])
        if o[i]==Number_level
            a1[i]=3
        end
        D1+=Q[i,a1[i]]
    end

    if !(3 in a1)
        return a0
    end

    if D0 > D1
        return a0
    else
        return a1
    end
end


    
    
    


decision (generic function with 1 method)

# $Q$-function
After obtaining $V^*(b)$ from policy, we calculate optimal $Q$-function by
$$Q^*(b,a)=R(b,a)+\gamma  \sum_{o'\in \mathcal{O}} \eta(o'|b,a)V^*(b')$$
where expected immediate reward of taking action $a$ in belief state $b$ is $$R(b,a)=\sum_{s\in \mathcal{S}}R(s,a) b(s),$$
and prior probability of observing $o'$ after taking action $a$ is $$\eta(o'|b,a)=\sum_{s'\in\mathcal{S}} P(o'|s',a)\sum_{s\in\mathcal{S}}T(s'|s,a)b(s)$$

Now $Q$ needs to be specified to each policy for each unit

In [53]:
function Qfnc(b)
    epsilon = 1e-4
    Q=zeros((NumberUnits,4)) 
    # NOTE: Q is defined for each unit (according to their beliefs), not state
    for i in 1:NumberUnits
        # Given unit index, specify the belief state, pomdp model, and policy for it
        belief=b[i]
        pomdp = Unit_pomdps[i]
        policy = Unit_policies[i]

        for a in 1:4
   
            Q_value=0
            for s in 1:Number_level
                Q_value+=belief[s] * POMDPs.reward(pomdp,s,a)
            end
            for o in 1:Number_level
            # the o here is the observation we expect to obtain at next time step, 
            # i.e., after taking the action we are deciding
            # so we don't need input current o
                o_prior = 0 
                for sp in 1:Number_level
                    p_sp=0
                    for s in 1:Number_level
                        p_sp+=pdf(transition(pomdp,s,a),sp) * belief[s]
                    end
                    p_sp=p_sp * Observation_matrix[sp,o]
                    o_prior+=p_sp
                end
     
                bp=update_belief(belief,a,o,i)
      
                if o_prior > epsilon
                    Q_value += gamma * o_prior * value(policy,bp)
                end
            end
            Q[i,a]=Q_value
        end
    end
    return Q
end
        
                
                

Qfnc (generic function with 1 method)

### Iteration

In [ ]:
using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],lamb=[],mean=[],std=[])

cost=[ 
    [0	-800	-100	-1200 -2000],
    [0	-1200	-100	-1200 -2000],
    [0	-1600	-100	-1200 -2000],
    [0	-2000	-100	-1200 -2000],
    [0  -2400   -100    -1200 -2000],
    [0  -2800   -100    -1200 -2000],
    [0  -3200   -100    -1200 -2000]
] 

for units in [20, 25, 30, 40, 50, 60]

    for K in 2:2:convert(Int64, floor((units+1)/2))  
        
    global NumberUnits=convert(Int64,units)
    global limit=convert(Int64,K)
    #for lambda in [1,0.9,0.75,0.5,0.1,0.075,0.05,0.01,0.0075,0.005,0.0001]
    for lambda in 1:1:1
    global lamb=lambda
    for c in cost
        tick()
        println(units)  
        print("K : ")
        println(K)
        global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
        println(c)  
        global NumberUnits=units
# POMDPs are defined
global Unit_pomdps = create_pomdps()
# CONTINUE to solve it and store the polices
global Unit_policies = solve_pomdps(Unit_pomdps)
# policy = solve(solver, multiunit)

trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads  for h in 1:trials
    s=repeat(1:1,NumberUnits)
    o = rand(Initialobs, NumberUnits)
    belief = [Observation_matrix[1,:] for j in 1:NumberUnits]
    R_sequence=[] 
    r=repeat(1:1,NumberUnits)*1.0

    # iteration
    for k in 1:simsteps
        a=decision(o,belief) # system-level decision making: action vector
        
        # SANITY CHECK
        # Update belief for each component individually
        # b=update_belief(b, a, o, multiunit) # what is the sequence of observing, updating belief, taking action?
        
        R_instant = 0.0
        global failed_components=0
        # When generating new state and new reward, we use true states
        for i in 1:NumberUnits # numerate over units
            if o[i]==Number_level # failure state is self-announcing
                failed_components+=1
            end
            s[i],r[i],o[i]=generative(s[i],a[i],i,234) #call generative function for underlying states
            # it might be better to use generative function to give o as well
            #o[i]=emission(s[i]) # observation generated by new underlying state
            belief[i]=update_belief(belief[i], a[i], o[i], i)
            # sum up single-component cost
            R_instant += r[i]
        end
        # add setup cost
        if (2 in a) ||(3 in a) ||(4 in a)
            R_instant += setup_cost
        end
        # add system failure penalty
        if failed_components >= limit
            R_instant += system_penalty
        end
        # record instant reward of current time step
        append!(R_sequence, R_instant)
    end
    R_discounted=0
    for k in 1:simsteps
        R_discounted=gamma * R_discounted+R_sequence[simsteps-k+1]
    end
    results[h]=R_discounted
end # end the 10000 trials

println(units)
            print("k : ")
            println(limit)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,lamb,mean(results),std(results)])
        
tock()

                    
                    
    end
    end        
end
end


20
K : 2
[0 -800 -100 -1200 -2000]


[ Info:  started timer at: 2025-03-08T14:23:23.150


20
k : 2
1
-21334.15356568197
2520.3332869499923
20
K : 2
[0 -1200 -100 -1200 -2000]


[ Info:         1854.3261379s: 30 minutes, 54 seconds, 326 milliseconds
[ Info:  started timer at: 2025-03-08T14:54:17.513


20
k : 2
1
-25225.667811798055
2797.3431896526217
20
K : 2
[0 -1600 -100 -1200 -2000]


...

[ Info:         5067.6565292s: 1 hour, 24 minutes, 27 seconds, 656 milliseconds
[ Info:  started timer at: 2025-03-19T06:34:06.910


60
k : 4
1
-57906.32237163122
3540.062976260419
60
K : 4
[0 -2400 -100 -1200 -2000]


[ Info:          5067.601253s: 1 hour, 24 minutes, 27 seconds, 601 milliseconds
[ Info:  started timer at: 2025-03-19T07:58:34.619


60
k : 4
1
-61897.85971467748
3706.156808755398
60
K : 4
[0 -2800 -100 -1200 -2000]


[ Info:         5102.7361667s: 1 hour, 25 minutes, 2 seconds, 736 milliseconds
[ Info:  started timer at: 2025-03-19T09:23:37.463


In [ ]:
df

In [ ]:
using CSV
CSV.write("Hetero CW-POMDP.csv", df)